In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 23 12:22:38 2023

@author: vishalr
"""
import numpy as np
import pandas as pd
import pickle

# change this to the folder where you store your data
data_dir = r"/content/sample_data/"

# each of the two data frames below have 20,000 rows, each corresponding to one sample from the original graph
# each sample consists of 21 nodes; node_labels contains the names of these 21 nodes of the form Vxyz
# each node is a DNA fragment of length 500 bases; so Vxyz coveres region [500*xyz, 500*xyz+500)
# adjacency_matrix has the flattended adjacecny matrix for each of these 20,000 samples
# so each row is of dimension 21*21 = 441

# node_labels = pd.read_pickle(data_dir+'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix')
# adjacency_matrix = pd.read_pickle(data_dir+'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot')

node_labels_chr2L = np.load(data_dir + 'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr2L = np.load(data_dir + 'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

node_labels_chr2R = np.load(data_dir + 'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr2R = np.load(data_dir + 'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

node_labels_chr3L = np.load(data_dir + 'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr3L = np.load(data_dir + 'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

node_labels_chr3R = np.load(data_dir + 'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot_sample_node_matrix.npy')
adjacency_matrix_chr3R = np.load(data_dir + 'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

# just seeing the size of input; doesn't really do anything
print("chr2L node labels shape:", node_labels_chr2L.shape)
print("chr2L adjacency matrix shape:", adjacency_matrix_chr2L.shape)
print("chr2R node labels shape:", node_labels_chr2R.shape)
print("chr2R adjacency matrix shape:", adjacency_matrix_chr2R.shape)
print("chr3L node labels shape:", node_labels_chr3L.shape)
print("chr3L adjacency matrix shape:", adjacency_matrix_chr3L.shape)
print("chr3R node labels shape:", node_labels_chr3R.shape)
print("chr3R adjacency matrix shape:", adjacency_matrix_chr3R.shape)


chr2L node labels shape: (20000, 21)
chr2L adjacency matrix shape: (20000, 442)
chr2R node labels shape: (20000, 21)
chr2R adjacency matrix shape: (20000, 442)
chr3L node labels shape: (20000, 21)
chr3L adjacency matrix shape: (20000, 442)
chr3R node labels shape: (20000, 21)
chr3R adjacency matrix shape: (20000, 442)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [3]:
# We assume that PyTorch is already installed
import torch
torchversion = torch.__version__

# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Visualization
import networkx as nx
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 26.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
# trying print the shape of the loaded array to get the number of samples
print(f'Number of samples: {adjacency_matrix_chr2L.shape[0]}')

Number of samples: 20000


In [5]:
#part1a: delete repeated node in adjacency matrix
import numpy as np

def remove_duplicates(node_labels, adj_matrix):
    unique_nodes = []
    unique_indices = []

    for idx, node in enumerate(node_labels):
        if node not in unique_nodes:
            unique_nodes.append(node)
            unique_indices.append(idx)

    unique_nodes = np.array(unique_nodes)
    sample_adj_matrix = adj_matrix[np.ix_(unique_indices, unique_indices)]

    return unique_nodes, sample_adj_matrix

# test whether it is functioning
sample_node_labels = np.array([1, 2, 2, 3, 4, 4])
sample_adj_matrix = np.array([
    [0, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1, 0]
])

cleaned_nodes, cleaned_adj_matrix = remove_duplicates(sample_node_labels, sample_adj_matrix)

print("Cleaned Nodes:", cleaned_nodes)
print("Cleaned Adjacency Matrix:\n", cleaned_adj_matrix)

Cleaned Nodes: [1 2 3 4]
Cleaned Adjacency Matrix:
 [[0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 1 0]]


In [8]:
#part 2a: to create a numpy array of all node labels, in file order
import numpy as np
all_node_indices = []

section_length = 60000
num_sections = 4

for section in range(num_sections):
    for i in range(1, section_length + 1):
        all_node_indices.append(i)

all_node_indices = np.array(all_node_indices)

# print to check the result
print("All node indices array:", all_node_indices)
print("Shape of all node indices array:", all_node_indices.shape)

#

All node indices array: [    1     2     3 ... 59998 59999 60000]
Shape of all node indices array: (240000,)


In [10]:
# part2b：indicate nodes' class
file_indices = []

section_length = 60000
num_sections = 4

for section in range(num_sections):
    file_indices.extend([section] * section_length)

file_indices = np.array(file_indices)

# Print to check the result
print("File indices array:", file_indices)
print("Shape of file indices array:", file_indices.shape)

# Print each section to verify
print("First section (should be 0):")
print(file_indices[:section_length])

print("Second section (should be 1):")
print(file_indices[section_length:2 * section_length])

print("Third section (should be 2):")
print(file_indices[2 * section_length:3 * section_length])

print("Fourth section (should be 3):")
print(file_indices[3 * section_length:])


File indices array: [0 0 0 ... 3 3 3]
Shape of file indices array: (240000,)
First section (should be 0):
[0 0 0 ... 0 0 0]
Second section (should be 1):
[1 1 1 ... 1 1 1]
Third section (should be 2):
[2 2 2 ... 2 2 2]
Fourth section (should be 3):
[3 3 3 ... 3 3 3]


In [13]:
#part 2c: edge list
import numpy as np
import scipy.sparse as sp

def extract_edges_from_subgraph(adjacency_matrix):
    num_nodes = 21
    edges = []

    for i in range(adjacency_matrix.shape[0]):
        # Reshape the adjacency matrix
        adj_matrix = adjacency_matrix[i, :num_nodes*num_nodes].reshape((num_nodes, num_nodes))

        for row in range(num_nodes):
            for col in range(row + 1, num_nodes):
                if adj_matrix[row, col] == 1:  # If there is an edge
                    edges.append(np.array([row, col]))

    return edges

# Extract edges from the adjacency matrices
edges_chr2L = extract_edges_from_subgraph(adjacency_matrix_chr2L)
edges_chr2R = extract_edges_from_subgraph(adjacency_matrix_chr2R)
edges_chr3L = extract_edges_from_subgraph(adjacency_matrix_chr3L)
edges_chr3R = extract_edges_from_subgraph(adjacency_matrix_chr3R)

# Print results to verify
print("Edges from chr2L:", edges_chr2L[:10])  # Print the first 10 edges as an example
print("Total edges in chr2L:", len(edges_chr2L))
print("Edges from chr2R:", edges_chr2R[:10])
print("Total edges in chr2R:", len(edges_chr2R))
print("Edges from chr3L:", edges_chr3L[:10])
print("Total edges in chr3L:", len(edges_chr3L))
print("Edges from chr3R:", edges_chr3R[:10])
print("Total edges in chr3R:", len(edges_chr3R))

all_edges = edges_chr2L + edges_chr2R + edges_chr3L + edges_chr3R

# Print the total number of edges
print("Total edges across all subgraphs:", len(all_edges))

# Save edges to a file
edges_file = 'edges.npy'
np.save(edges_file, all_edges)

# Load edges from the file to verify
loaded_edges = np.load(edges_file, allow_pickle=True)
print(f"Total edges loaded: {len(loaded_edges)}")
print(f"First 10 edges: {loaded_edges[:10]}")

Edges from chr2L: [array([0, 1]), array([0, 2]), array([1, 2]), array([2, 3]), array([3, 4]), array([4, 5]), array([4, 7]), array([4, 9]), array([ 4, 11]), array([5, 6])]
Total edges in chr2L: 722875
Edges from chr2R: [array([0, 1]), array([1, 2]), array([1, 4]), array([1, 7]), array([1, 9]), array([2, 3]), array([2, 5]), array([2, 6]), array([2, 8]), array([ 2, 10])]
Total edges in chr2R: 754087
Edges from chr3L: [array([0, 1]), array([1, 2]), array([2, 3]), array([2, 5]), array([2, 6]), array([2, 8]), array([3, 4]), array([3, 6]), array([4, 5]), array([5, 6])]
Total edges in chr3L: 679556
Edges from chr3R: [array([0, 1]), array([1, 2]), array([1, 7]), array([1, 8]), array([2, 3]), array([2, 5]), array([2, 7]), array([3, 4]), array([3, 6]), array([3, 7])]
Total edges in chr3R: 718848
Total edges across all subgraphs: 2875366
Total edges loaded: 2875366
First 10 edges: [[ 0  1]
 [ 0  2]
 [ 1  2]
 [ 2  3]
 [ 3  4]
 [ 4  5]
 [ 4  7]
 [ 4  9]
 [ 4 11]
 [ 5  6]]


In [15]:
# part 2d: Function to generate class labels for edges
def generate_edge_classes(adjacency_matrix, class_label):
    num_nodes = 21
    edge_classes = []

    for i in range(adjacency_matrix.shape[0]):
        adj_matrix = adjacency_matrix[i, :num_nodes*num_nodes].reshape((num_nodes, num_nodes))

        for row in range(num_nodes):
            for col in range(row + 1, num_nodes):
                if adj_matrix[row, col] == 1:  # If there is an edge
                    edge_classes.append(class_label)

    return edge_classes

# Load the subgraph files
data_dir = '/content/sample_data/'
adjacency_matrix_chr2L = np.load(data_dir + 'df_chr2L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')
adjacency_matrix_chr2R = np.load(data_dir + 'df_chr2R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')
adjacency_matrix_chr3L = np.load(data_dir + 'df_chr3L_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')
adjacency_matrix_chr3R = np.load(data_dir + 'df_chr3R_drosophila_ChIA_Drop_0.1_PASS_20000_MCMC_pivot.npy')

edges_chr2L = extract_edges_from_subgraph(adjacency_matrix_chr2L)
edges_chr2R = extract_edges_from_subgraph(adjacency_matrix_chr2R)
edges_chr3L = extract_edges_from_subgraph(adjacency_matrix_chr3L)
edges_chr3R = extract_edges_from_subgraph(adjacency_matrix_chr3R)

edge_classes_chr2L = generate_edge_classes(adjacency_matrix_chr2L, 0)
edge_classes_chr2R = generate_edge_classes(adjacency_matrix_chr2R, 1)
edge_classes_chr3L = generate_edge_classes(adjacency_matrix_chr3L, 2)
edge_classes_chr3R = generate_edge_classes(adjacency_matrix_chr3R, 3)

all_edges = edges_chr2L + edges_chr2R + edges_chr3L + edges_chr3R
all_edge_classes = edge_classes_chr2L + edge_classes_chr2R + edge_classes_chr3L + edge_classes_chr3R

# Save classes to files
edge_classes_file = '/content/edge_classes.npy'
np.save(edge_classes_file, all_edge_classes)

# Load edges and their classes from the files to verify
loaded_edge_classes = np.load(edge_classes_file, allow_pickle=True)
print(f"Total edge classes loaded: {len(loaded_edge_classes)}")
print(f"First 10 edge classes: {loaded_edge_classes[:10]}")
print(f"Last 10 edge classes: {loaded_edge_classes[-10:]}")


Total edge classes loaded: 2875366
First 10 edge classes: [0 0 0 0 0 0 0 0 0 0]
Last 10 edge classes: [3 3 3 3 3 3 3 3 3 3]


In [ ]:
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader
from torch_geometric.utils import from_scipy_sparse_matrix
import scipy.sparse as sp
import numpy as np


def create_graph_data(adjacency_matrix, node_labels, label_number):
    graphs = []
    for i in range(adjacency_matrix.shape[0]):
        row = adjacency_matrix[i]
        labels = node_labels[i]
        assert len(row) == 21 * 21 + 1, f"Each row should have 21*21 + 1 elements, but got {len(row)} elements."
        row = row[:-1]  # remove -1 row
        assert len(row) == 21 * 21, f"Each row should have 21*21 elements after removing -1 column, but got {len(row)} elements."
        adj_matrix = row.reshape((21, 21))
        unique_nodes = []
        unique_indices = []
        for idx, node in enumerate(labels):
            if node not in unique_nodes:
                unique_nodes.append(node)
                unique_indices.append(idx)
        labels = np.array(unique_nodes)
        adj_matrix = adj_matrix[np.ix_(unique_indices, unique_indices)]
        edge_index, edge_attr = from_scipy_sparse_matrix(sp.coo_matrix(adj_matrix))
        x = torch.tensor(labels, dtype=torch.float).view(-1, 1)
        y = torch.tensor([label_number], dtype=torch.long)
        graph_data = Data(x=x, edge_index=edge_index, y=y)
        graphs.append(graph_data)
    return graphs

worklist_node_labels = [node_labels_chr2L, node_labels_chr2R, node_labels_chr3L, node_labels_chr3R]
worklist_adjacency_matrix = [adjacency_matrix_chr2L, adjacency_matrix_chr2R, adjacency_matrix_chr3L, adjacency_matrix_chr3R]

dataset = []
for label_number in range(len(worklist_adjacency_matrix)):
    adj_matrix = worklist_adjacency_matrix[label_number]
    node_labels = worklist_node_labels[label_number]
    graphs = create_graph_data(adj_matrix, node_labels, label_number)
    dataset.extend(graphs)

class MyDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None, pre_transform=None):
        self.data_list = data_list
        super(MyDataset, self).__init__(None, transform, pre_transform)
        self.data, self.slices = self.collate(data_list)

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

my_dataset = MyDataset(dataset)

num_node_features = my_dataset[0].x.size(1)
num_classes = len(set([data.y.item() for data in my_dataset]))


# Calculate num_node_features and num_classes
num_node_features = my_dataset[0].x.size(1)
num_classes = len(set([data.y.item() for data in my_dataset]))

# Split the dataset into training, validation, and test sets
train_dataset = my_dataset[:int(len(my_dataset) * 0.8)]
val_dataset = my_dataset[int(len(my_dataset) * 0.8):int(len(my_dataset) * 0.9)]
test_dataset = my_dataset[int(len(my_dataset) * 0.9):]

print(f'Training set   = {len(train_dataset)} graphs')
print(f'Validation set = {len(val_dataset)} graphs')
print(f'Test set       = {len(test_dataset)} graphs')

# Create mini-batches
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Verify if everything works properly
print('\nTrain loader:')
for i, subgraph in enumerate(train_loader):
    print(f' - Subgraph {i}:')
    print(f'  Node labels (x): {subgraph.x}')
    print(f'  Labels (y): {subgraph.y}')
    break  # Print only the first batch

print('\nValidation loader:')
for i, subgraph in enumerate(val_loader):
    print(f' - Subgraph {i}:')
    print(f'  Node labels (x): {subgraph.x}')
    print(f'  Labels (y): {subgraph.y}')
    break  # Print only the first batch

print('\nTest loader:')
for i, subgraph in enumerate(test_loader):
    print(f' - Subgraph {i}:')
    print(f'  Node labels (x): {subgraph.x}')
    print(f'  Labels (y): {subgraph.y}')
    break  # Print only the first batch

    # Create the custom dataset
my_dataset = MyDataset(dataset)




Training set   = 64000 graphs
Validation set = 8000 graphs
Test set       = 8000 graphs

Train loader:
 - Subgraph 0:
  Node labels (x): tensor([[33208.],
        [19267.],
        [37250.],
        ...,
        [23387.],
        [15986.],
        [16159.]])
  Labels (y): tensor([0, 1, 2, 1, 0, 2, 2, 1, 2, 1, 0, 2, 2, 2, 1, 0, 2, 2, 0, 0, 3, 3, 1, 2,
        0, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 1, 2, 0, 0, 1, 1, 0, 0, 2, 2, 1, 3, 2,
        0, 1, 2, 1, 0, 2, 2, 0, 0, 2, 0, 0, 3, 0, 0, 0])

Validation loader:
 - Subgraph 0:
  Node labels (x): tensor([[51077.],
        [39538.],
        [33879.],
        ...,
        [34080.],
        [34069.],
        [34096.]])
  Labels (y): tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

Test loader:
 - Subgraph 0:
  Node labels (x): tensor([[51678.],
        [42148.],
        [19282.],
 

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout
from torch_geometric.nn import GCNConv, GINConv
from torch_geometric.nn import global_mean_pool, global_add_pool


# class GCN(torch.nn.Module):
#     """GCN"""
#     def __init__(self, dim_h):
#         super(GCN, self).__init__()
#         self.conv1 = GCNConv(dataset.num_node_features, dim_h)
#         self.conv2 = GCNConv(dim_h, dim_h)
#         self.conv3 = GCNConv(dim_h, dim_h)
#         self.lin = Linear(dim_h, dataset.num_classes)

#     def forward(self, x, edge_index, batch):
#         # Node embeddings
#         h = self.conv1(x, edge_index)
#         h = h.relu()
#         h = self.conv2(h, edge_index)
#         h = h.relu()
#         h = self.conv3(h, edge_index)

#         # Graph-level readout
#         hG = global_mean_pool(h, batch)

#         # Classifier
#         h = F.dropout(hG, p=0.5, training=self.training)
#         h = self.lin(h)

#         return hG, F.log_softmax(h, dim=1)

class GIN(torch.nn.Module):
    """GIN"""
    def __init__(self, dim_h):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(Linear(my_dataset.num_node_features, dim_h),
                       BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv2 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.conv3 = GINConv(
            Sequential(Linear(dim_h, dim_h), BatchNorm1d(dim_h), ReLU(),
                       Linear(dim_h, dim_h), ReLU()))
        self.lin1 = Linear(dim_h*3, dim_h*3)
        self.lin2 = Linear(dim_h*3, my_dataset.num_classes)

    def forward(self, x, edge_index, batch):
        # Node embeddings
        h1 = self.conv1(x, edge_index)
        h2 = self.conv2(h1, edge_index)
        h3 = self.conv3(h2, edge_index)

        # Graph-level readout
        h1 = global_add_pool(h1, batch)
        h2 = global_add_pool(h2, batch)
        h3 = global_add_pool(h3, batch)

        # Concatenate graph embeddings
        h = torch.cat((h1, h2, h3), dim=1)

        # Classifier
        h = self.lin1(h)
        h = h.relu()
        h = F.dropout(h, p=0.5, training=self.training)
        h = self.lin2(h)

        return h, F.log_softmax(h, dim=1)

gin = GIN(dim_h=32)

In [ ]:
def train(model, loader):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.01,
                                      weight_decay=0.01)
    epochs = 5

    model.train()
    for epoch in range(epochs+1):
        #print(epoch)
        total_loss = 0
        acc = 0
        val_loss = 0
        val_acc = 0

        # Train on batches
        for data in loader:
          optimizer.zero_grad()
          _, out = model(data.x, data.edge_index, data.batch)
          loss = criterion(out, data.y)
          total_loss += loss / len(loader)
          acc += accuracy(out.argmax(dim=1), data.y) / len(loader)
          loss.backward()
          optimizer.step()

          # Validation
          val_loss, val_acc = test(model, val_loader)

          # print per epoch to see if the accuracy goes up!
        print(f'Epoch {epoch:>0} | Train Loss: {total_loss:.2f} '
            f'| Train Acc: {acc*100:>5.2f}% '
            f'| Val Loss: {val_loss:.2f} '
            f'| Val Acc: {val_acc*100:.2f}%')


    test_loss, test_acc = test(model, test_loader)
    print(f'Test Loss: {test_loss:.2f} | Test Acc: {test_acc*100:.2f}%')

    return model

@torch.no_grad()
def test(model, loader):
    criterion = torch.nn.CrossEntropyLoss()
    model.eval()
    loss = 0
    acc = 0

    for data in loader:
        _, out = model(data.x, data.edge_index, data.batch)
        loss += criterion(out, data.y) / len(loader)
        acc += accuracy(out.argmax(dim=1), data.y) / len(loader)

    return loss, acc

def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

gin = train(gin, train_loader)

Epoch 0 | Train Loss: 1.12 | Train Acc: 45.11% | Val Loss: 2.34 | Val Acc: 16.16%
Epoch 1 | Train Loss: 1.10 | Train Acc: 46.14% | Val Loss: 1.89 | Val Acc: 33.35%
Epoch 2 | Train Loss: 1.09 | Train Acc: 46.51% | Val Loss: 1.75 | Val Acc: 38.32%
Epoch 3 | Train Loss: 1.10 | Train Acc: 46.46% | Val Loss: 1.81 | Val Acc: 32.44%
Epoch 4 | Train Loss: 1.09 | Train Acc: 46.89% | Val Loss: 1.37 | Val Acc: 49.36%
Epoch 5 | Train Loss: 1.09 | Train Acc: 46.80% | Val Loss: 1.61 | Val Acc: 38.95%
Test Loss: 1.59 | Test Acc: 40.66%


In [ ]:

label_mapping = {0: '2L', 1: '2R', 2: '3L', 3: '3R'}


import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx


fig, ax = plt.subplots(4, 4, figsize=(16, 16))
fig.suptitle('GIN - Graph classification')

for i, data in enumerate(dataset[1113-16:]):
    _, out = gin(data.x, data.edge_index, data.batch)
    predicted_class = out.argmax(dim=1).item()
    actual_class = data.y.item()
    color = "green" if predicted_class == actual_class else "red"

    ix = np.unravel_index(i, ax.shape)
    ax[ix].axis('off')
    G = to_networkx(dataset[1113-16+i], to_undirected=True)
    nx.draw_networkx(G,
                     pos=nx.spring_layout(G, seed=0),
                     with_labels=False,
                     node_size=150,
                     node_color=color,
                     width=0.8,
                     ax=ax[ix])

    classification_text = f"Predicted: {label_mapping[predicted_class]}, Actual: {label_mapping[actual_class]}"
    ax[ix].text(0.5, 0.9, classification_text, horizontalalignment='center', verticalalignment='center', transform=ax[ix].transAxes, color='black')

plt.show()


ModuleNotFoundError: No module named 'torch_geometric'